# GridSearch + Validation croisée + Hyperparamètres

## 0) Imports + dataset

In [ ]:
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [12]:
# Dataset réel
X, y = load_wine(return_X_y=True)

# Split train/test (test = évaluation finale)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train:", X_train.shape, "Test:", X_test.shape)

Train: (142, 13) Test: (36, 13)


## 1) Rappel : paramètres vs hyperparamètres (Markdown)

- **Paramètres :** appris pendant fit (ex: poids d’un modèle linéaire).

- **Hyperparamètres :** choisis avant fit (ex: k pour KNN, C pour Logistic Regression).

## 2) Pourquoi la Cross-Validation ?

In [13]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

KFold mais Stratified car en classification, on garde la proportion des classes similaire dans chaque fold.

## 3) Exemple 1 — KNN : chercher le meilleur k avec GridSearchCV

### 3.1 Pipeline (scaling + modèle)

In [14]:
pipe_knn = Pipeline([
    ("scaler", StandardScaler()),
    ("model", KNeighborsClassifier())
])

### 3.2 Grille d’hyperparamètres

In [15]:
param_grid_knn = {
    "model__n_neighbors": range(1, 31)
}

### 3.3 GridSearchCV

In [16]:
grid_knn = GridSearchCV(
    estimator=pipe_knn,
    param_grid=param_grid_knn,
    cv=cv,
    scoring="accuracy",
    n_jobs=-1
)

grid_knn.fit(X_train, y_train)

print("Meilleurs hyperparamètres:", grid_knn.best_params_)
print("Meilleur score CV:", round(grid_knn.best_score_, 3))
print("Score test final:", round(grid_knn.score(X_test, y_test), 3))

Meilleurs hyperparamètres: {'model__n_neighbors': 29}
Meilleur score CV: 0.979
Score test final: 0.972


#### Point clés :

- GridSearch essaie tous les k

- Pour chaque k, il fait une CV (5 entraînements)

- Il moyenne les scores

- Il garde le meilleur

- Puis il évalue une seule fois sur le test

## 4) Exemple 2 — Logistic Regression : chercher le meilleur C

### 4.1 Pipeline

In [ ]:
pipe_lr = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter=5000))
])

### 4.2 Grille

In [ ]:
param_grid_lr = {
    "model__C": [0.01, 0.1, 1, 10, 100]
}

### 4.3 GridSearch

In [ ]:
grid_lr = GridSearchCV(
    estimator=pipe_lr,
    param_grid=param_grid_lr,
    cv=cv,
    scoring="accuracy",
    n_jobs=-1
)

grid_lr.fit(X_train, y_train)

print("Meilleurs hyperparamètres:", grid_lr.best_params_)
print("Meilleur score CV:", round(grid_lr.best_score_, 3))
print("Score test final:", round(grid_lr.score(X_test, y_test), 3))

### Ce que fait réellement GridSearchCV

Pour chaque combinaison d’hyperparamètres :

- Il applique la validation croisée

- Il calcule le score moyen

- Il compare les scores

- Il sélectionne le meilleur

- Il réentraîne sur tout le train avec la meilleure config

**Donc :**

> GridSearchCV = exploration systématique + validation croisée automatisée.


## 5) Comparer 2 modèles proprement (même protocole)

### 5.1 Comparaison des GridSearch directement

In [ ]:
grids_dict = {
    "KNN": grid_knn,
    "LogReg": grid_lr
}

In [ ]:
results = {}

for name, grid in grids_dict.items():
    results[name] = {
        "best_params": grid.best_params_,
        "cv_score": grid.best_score_,
        "test_score": grid.score(X_test, y_test)
    }

results

### 5.2 Comparaison des modèles (pipeline et grille à comparer)

In [ ]:
models = {
    # nom_du_model : (pipeline, grille)
    "KNN": (pipe_knn, param_grid_knn),
    "LogReg": (pipe_lr, param_grid_lr)
}

results = {}

for name, (pipe, grid) in models.items():
    gs = GridSearchCV(pipe, grid, cv=cv, scoring="accuracy", n_jobs=-1)
    gs.fit(X_train, y_train)

    results[name] = {
        "best_params": gs.best_params_,
        "cv_score": gs.best_score_,
        "test_score": gs.score(X_test, y_test)
    }

results

{'KNN': {'best_params': {'model__n_neighbors': 29},
  'cv_score': np.float64(0.9788177339901478),
  'test_score': 0.9722222222222222},
 'LogReg': {'best_params': {'model__C': 1},
  'cv_score': np.float64(0.9790640394088669),
  'test_score': 0.9722222222222222}}

**Ici on observe :**

- Comparaison équitable

- Même jeu de données

- Même CV

- Même métrique

- C’est une vraie logique “entreprise”.

## Option (facultative) — RandomizedSearchCV

### Quand la grille est énorme, on échantillonne au hasard.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

rand_knn = RandomizedSearchCV(
    estimator=pipe_knn,
    param_distributions={"model__n_neighbors": randint(1, 51)},
    n_iter=20,
    cv=cv,
    scoring="accuracy",
    n_jobs=-1,
    random_state=42
)

rand_knn.fit(X_train, y_train)

print("Best params (random):", rand_knn.best_params_)
print("Best CV score:", round(rand_knn.best_score_, 3))
print("Test score:", round(rand_knn.score(X_test, y_test), 3))

## Récap

- CV : robuste, réduit l’effet “split chanceux”

- GridSearchCV : teste toutes les combinaisons

- RandomizedSearchCV : teste un nombre limité de combinaisons aléatoires

- Pipeline : garantit que le preprocessing est fait correctement dans chaque fold

- Test set : une seule fois, à la fin

## Message fondamental (important pour le DL)

**En ML, le cœur du travail n’est pas le modèle. C’est la méthode de validation.**

**En Deep Learning, ce sera exactement pareil :**

- On choisira learning rate

- On choisira architecture

- On choisira nombre de couches

- On regardera la courbe validation

- La validation est déjà introduite.